FETCH HISTORIC DATA FROM YFINANCE

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import date

start_date = "2020-01-01"
end_date = "2025-03-15"


df = pd.read_csv("../datasets/tickers.csv")
symbols = df["symbol"].tolist()

data = pd.DataFrame()

for symbol in symbols:
    try:
        ticker_data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=True)
        
        ticker_data.columns = ticker_data.columns.get_level_values(0)
        ticker_data = ticker_data.reset_index()
        ticker_data = ticker_data.rename(columns={"price": "id"})
        ticker_data['symbol'] = symbol
        
        data = pd.concat([data, ticker_data], ignore_index=True)
    except Exception as e:
            print(f"Fetch error {symbol}: {e}")

print(data)

In [ ]:
data.to_csv('../datasets/yahoo_historic_data.csv',index=False)

HISTORIC DATASET REVIEW

<small>Restart jupyter notebook</small>

In [ ]:
import pandas as pd

df = pd.read_csv('../datasets/yahoo_historic_data.csv')
df

In [ ]:
#remove stocks that have less than 1000 days of history
pocet_radku_alt = df["symbol"].value_counts()

symbols_to_keep = pocet_radku_alt[pocet_radku_alt >= 1000].index
df = df[df["symbol"].isin(symbols_to_keep)]

In [ ]:
unique_symbols = df["symbol"].nunique()
print("Počet unikátních symbolů:", unique_symbols)

rows_on_date = df[df["Date"] == "2025-03-14"].shape[0]
print("Počet řádků s datem 2025-03-14:", rows_on_date)

#rows with empty values
print(df.isnull().sum())


In [ ]:
df.columns = df.columns.str.lower()
df

In [ ]:

rearrange_columns = ['date','symbol','open','high','low','close','volume']

df = df[rearrange_columns]
df.to_csv('../datasets/hist_data.csv', index=False)

In [ ]:
#stocks that have at least 1000 days of history

symbols = df['symbol'].unique()

df_symbols = pd.DataFrame(symbols, columns=['symbol'])

df_symbols.to_csv('../datasets/new_tickers_list.csv', index=False)

FETCH COMPANY INFORMATION DATA FROM YFINANCE

<small>Restart jupyter notebook</small>


In [ ]:
import pandas as pd
import yfinance as yf
from datetime import date

start_date = "2020-01-01"
end_date = date.today().strftime("%Y-%m-%d")  # ve formátu YYYY-MM-DD

# Načtení seznamu tickerů ze souboru CSV
df = pd.read_csv("../datasets/new_tickers_list.csv")
symbols = df["symbol"].tolist()

data = pd.DataFrame()

for symbol in symbols:
    try:
        stock = yf.Ticker(symbol)
        info = stock.info

        company_data = {
                "symbol": symbol,
                "short_name": info.get("shortName"),
                "region": info.get("region"),
                "exchange": info.get("exchange"),
                "exchange_timezone": info.get("exchangeTimezoneShortName"),
                "market_cap": info.get("marketCap"),
                "beta": info.get("beta"),
        }
        new_df = pd.DataFrame([company_data])
        
        data = pd.concat([data, new_df], ignore_index=True)
    except Exception as e:
            print(f"Fetch error {symbol}: {e}")

print(data)

In [ ]:
data.to_csv('../datasets/comp_data.csv',index=False)

COMPANY INFO DATASET REVIEW

<small>Restart jupyter notebook</small>

In [ ]:
import pandas as pd

df = pd.read_csv('../datasets/comp_data.csv')

In [ ]:
df.head()

In [ ]:
print(df.isnull().sum())

SEEED DATA INTO DATABASE

<small>Restart jupyter notebook</small>

In [ ]:
import os
import psycopg2
from dotenv import load_dotenv

load_dotenv()


DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)


cursor = conn.cursor()

print("✅ Connect to database!")


In [ ]:
import pandas as pd

df_company = pd.read_csv("../datasets/comp_data.csv")
df_history = pd.read_csv("../datasets/hist_data.csv")

In [ ]:
import numpy as np

sql_companies = """
    INSERT INTO companies (symbol, short_name, region, exchange, exchange_timezone,
                        market_cap, beta )
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

sql_historic_data = """
    INSERT INTO daily_historic_data (date, symbol, open, high, low, close,
                        volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""



def prepare_values_for_sql(df):
    df = df.astype(object)
    
    df = df.where(pd.notna(df), None)
    
    values = [
        tuple(x.item() if isinstance(x, (np.integer, np.floating)) else x for x in row)
        for row in df.to_records(index=False)
    ]
    
    return values

cursor.executemany(sql_companies, prepare_values_for_sql(df_company))
cursor.executemany(sql_historic_data, prepare_values_for_sql(df_history))

conn.commit()

In [ ]:
cursor.close()
conn.close()